In [ ]:
from pathlib import Path

import torch

from rtnls_inference import (
    HeatmapRegressionEnsemble,
    SegmentationEnsemble,
)

## Segmentation of preprocessed images

Here we segment images preprocessed using 0_preprocess.ipynb


In [ ]:
ds_path = Path("../samples/fundus")

# input folders. these are the folders where we stored the preprocessed images
rgb_path = ds_path / "rgb"
ce_path = ds_path / "ce"

# these are the output folders for:
av_path = ds_path / "av"                # artery-vein segmentations
discs_path = ds_path / "discs"          # optic disc segmentations
overlays_path = ds_path / "overlays"    # optional overlay visualizations

device = torch.device("cuda:0")         # device to use for inference

In [ ]:
rgb_paths = sorted(list(rgb_path.glob("*.png")))
ce_paths = sorted(list(ce_path.glob("*.png")))
paired_paths = list(zip(rgb_paths, ce_paths))

In [ ]:
paired_paths[0]         # important to make sure that the paths are paired correctly

### Artery-vein segmentation


In [ ]:
av_ensemble = SegmentationEnsemble.from_release("av_july24.pt").to(device)
av_ensemble.predict_preprocessed(paired_paths, dest_path=av_path, num_workers=2)

### Disc segmentation


In [ ]:
disc_ensemble = SegmentationEnsemble.from_release("disc_july24.pt").to(device)
disc_ensemble.predict_preprocessed(
    paired_paths, dest_path=discs_path, num_workers=2
)

### Fovea detection


In [ ]:
fovea_ensemble = HeatmapRegressionEnsemble.from_release("fovea_july24.pt").to(device)
# note: this model does not use contrast enhanced images
df = fovea_ensemble.predict_preprocessed(paired_paths, num_workers=2)
df.columns = ["mean_x", "mean_y"]
df.to_csv(ds_path / "fovea.csv")

In [ ]:
df

### Plotting the retinas (optional)

This will only work if you ran all the models and stored the outputs using the same folder/file names as above


In [ ]:
from vascx.utils.loader import RetinaLoader
from rtnls_enface.utils.plotting import plot_gridfns

loader = RetinaLoader.from_folder(ds_path)

In [ ]:
plot_gridfns([ret.plot for ret in loader[:6]])

### Storing visualizations (optional)


In [ ]:
if not overlays_path.exists():
    overlays_path.mkdir()
for ret in loader:
    fig, _ = ret.plot()
    fig.savefig(overlays_path / f"{ret.id}.png", bbox_inches="tight", pad_inches=0)